In [ ]:
# Remember that CSV SINK is Loaded from Table module therefore it can use register_table_sink 
# however KafkaSource is loaded from datastream module therefore it cannot use register_table_source

In [1]:
from pyflink.table import TableEnvironment, EnvironmentSettings, DataTypes, StreamTableEnvironment

In [2]:
from pyflink.datastream.connectors.kafka import KafkaSource, KafkaOffsetsInitializer

In [3]:
from pyflink.datastream.connectors import KafkaSource

In [4]:
from pyflink.common.serialization import SimpleStringSchema

In [5]:
from pyflink.datastream.formats.json import JsonRowDeserializationSchema

In [6]:
from pyflink.datastream import StreamExecutionEnvironment

In [7]:
from pyflink.common import Types

In [8]:
from pyflink.common.watermark_strategy import WatermarkStrategy

In [9]:
from pyflink.common.configuration import Configuration

In [10]:
from pyflink.common.restart_strategy import RestartStrategies

In [11]:
from pyflink.table import CsvTableSink

In [12]:
stream_env = StreamExecutionEnvironment.get_execution_environment()

In [13]:
stream_settings = EnvironmentSettings.new_instance()\
                                    .in_streaming_mode()\
                                    .build()

In [14]:
stream_tbl_env = StreamTableEnvironment.create(stream_execution_environment=stream_env,environment_settings=stream_settings)

In [15]:
stream_tbl_env.get_config().get_configuration().set_string("parallelism.default", "1")

In [16]:
# Put Jars in: D:\Users\kritagya\anaconda3\envs\PyFlink\Lib\site-packages\pyflink

In [17]:
# stream_tbl_env.get_config().set('pipeline.jars','file:///C:/Users/krita/Udemy/PyFlink/flink-sql-connector-kafka-1.17.1.jar')

In [18]:
# stream_env.add_jars('pipeline.jars','file:///C:/Users/krita/Udemy/PyFlink/flink-sql-connector-kafka-1.17.1.jar')

In [19]:
json_types = [Types.STRING(),Types.STRING(),Types.INT(),Types.DOUBLE(),Types.SQL_DATE()]

In [20]:
json_names = ['seller_id','product','quantity','product_price','sales_date']

In [21]:
json_schema = Types.ROW_NAMED(field_names=json_names,field_types=json_types)

In [22]:
json_deserialization_schema = JsonRowDeserializationSchema.Builder()\
                                                        .type_info(type_info=json_schema)\
                                                        .ignore_parse_errors()\
                                                        .build()

In [23]:
kafka_source = KafkaSource.builder() \
    .set_bootstrap_servers('localhost:9092') \
    .set_topics("productsales") \
    .set_group_id("source-demo") \
    .set_starting_offsets(KafkaOffsetsInitializer.earliest()) \
    .set_value_only_deserializer(deserialization_schema=json_deserialization_schema) \
    .build()

In [24]:
kafka_stream_tbl = stream_tbl_env.from_data_stream(data_stream=stream_env.from_source(source=kafka_source,watermark_strategy=WatermarkStrategy.no_watermarks(),source_name='kafka_tbl_source'))

In [25]:
csv_types = [DataTypes.STRING(),DataTypes.STRING(),DataTypes.INT(),DataTypes.DOUBLE(),DataTypes.DATE()]

In [26]:
csv_names = ['seller_id','product','quantity','product_price','sales_date']

In [28]:
csv_sink = CsvTableSink(field_names=csv_names,field_types=csv_types,path='./kafka_local_sales.csv',num_files=1)

In [29]:
stream_tbl_env.register_table_sink(name='kafka_tbl_sink',table_sink=csv_sink)

In [30]:
kafka_stream_tbl.execute_insert(table_path_or_descriptor='kafka_tbl_sink')

In [ ]:
# Remember as the CSV is in stream mode therefore any on coming data from source will be directly stored in CSV